In [1]:
from ultralytics import YOLO
import os
import cv2
import numpy as np
import cv2
import zipfile
import json
import pathlib
import shutil
import json
import os
import shutil
import yaml
import matplotlib.pyplot as plt


In [5]:
from tkinter import Tk
from tkinter.filedialog import askdirectory

# Hide the main Tkinter window
Tk().withdraw()
# Print the selected folder
print(f"Please select the folder from the pop up, it will appear after you minimize this window:")
# Ask the user to select the folder with custom text
home_folder = askdirectory(title="Select the Project_submission folder")

# Check if a folder was selected
if not home_folder:
    print("No folder selected. Exiting.")
    exit()

# Print the selected folder or use it in your script
print(f"Selected folder: {home_folder}")


Please select the folder from the pop up, it will appear after you minimize this window:
Selected folder: D:/Others/Project_Submission_ALL_DATA


In [2]:
import shutil

def delete_folder(folder_path):

  # Check if the path exists
  if not os.path.exists(folder_path):
        print("Folder does not exist.")

  else:
      # Remove the folder and its contents
      shutil.rmtree(folder_path)
      print("The folder removed successfully ....")
# Example usage
#delete_folder("Cattle_Data_4_classes")

## 🖼️ `combine_masks(masks)`

### Overview
The `combine_masks` function takes multiple binary masks (e.g., predicted object masks) and combines them into a single mask using a **bitwise OR** operation. This is useful when you want a unified mask representing all objects in an image.


### Returns
- `np.ndarray`: A single combined mask of shape `(height, width)` representing the union of all input masks.

---



In [3]:
# Function to combine masks
def combine_masks(masks):
    combined_mask = np.zeros_like(masks[0])  # Initialize combined mask with zeros

    # Iterate through each mask and perform bitwise OR operation
    for mask in masks:
        combined_mask = np.bitwise_or(combined_mask, mask)

    return combined_mask

# Example usage
# Assuming masks is a numpy array of shape (num_mask, height, width) containing the masks
# mask = combine_masks(masks_array)


## 🖼️ `get_final_image(mask, original_image)`

### Overview
The `get_final_image` function applies a binary mask to an original image, producing a final image where only the masked regions are visible. It ensures the mask matches the image size and converts it to a 3-channel format for proper visualization.


### Returns
- `np.ndarray`: The masked image with the same dimensions as `original_image`. Pixels outside the mask are removed, and the result is in RGB format.




In [4]:
def get_final_image(mask , original_image):
  image = original_image

    # Ensure mask and image have the same dimensions
  mask_resized = cv2.resize(mask, (image.shape[1], image.shape[0])) * 255

  # Threshold the mask
  _, thresh = cv2.threshold(mask_resized, 127, 255, cv2.THRESH_BINARY)

  plt.imshow(thresh, cmap='gray')  # Visualize the thresholded mask

  # Convert the mask to 3 channels
  thresh_rgb = cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)

  # Bitwise AND of the image and mask
  final_image = cv2.bitwise_and(image, thresh_rgb)

  return final_image[:,:,::-1]

## 🖼️ `predict_images(input_folder, output_folder, model_weights)`



### ⚙️ Parameters

| Parameter | Type | Description |
|-----------|------|-------------|
| `input_folder` | `str` | Path to the folder containing input images. The function will iterate through all subfolders and process `.jpg` or `.png` files. |
| `output_folder` | `str` | Path where the processed images will be saved. Subfolders are created automatically if they do not exist. |
| `model_weights` | `str` | Path to the trained YOLO model weights file (`.pt`) used for segmentation predictions. |

---

### 📤 Returns
This function does **not** return any values.  
The processed images are saved directly in the `output_folder`, preserving the original subfolder structure.

### 📤 Notes
Some images may not produce a mask, so for such cases, you need to run this function on a single image and lower the confidence threshold. With the reduced confidence, the model will generate the output.

In [ ]:

import os
from PIL import Image

# Predict images using YOLO and save masked results
def predict_images(input_folder, output_folder, model_weights):
    yolo = YOLO(model_weights)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for folders in os.listdir(input_folder):
        folder_path = os.path.join(input_folder, folders)
        folder_out = os.path.join(output_folder, folders)
        os.makedirs(folder_out, exist_ok=True)

        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.jpg', '.png')):
                image_path = os.path.join(folder_path, filename)

                # Predict
                prediction = yolo.predict(image_path, conf=0.05, verbose=False)
                if not prediction:
                    print(f"No prediction for image: {image_path}")
                    continue

                new_result = prediction[0]
                if new_result is None or new_result.masks is None:
                    print(f"No masks detected for image: {image_path}")
                    continue

                extracted_masks = new_result.masks.data
                if len(extracted_masks) == 0:
                    print(f"No mask data for image: {image_path}")
                    continue

                masks_array = extracted_masks.cpu().numpy().astype('uint8')
                mask = combine_masks(masks_array)
                final_image = get_final_image(mask, new_result.orig_img)
                final_image = Image.fromarray(final_image)

                output_path = os.path.join(folder_out, filename)
                final_image.save(output_path)
                print(f"Saved masked image: {output_path}")

# Example usage
input_folder = os.path.join(home_folder, "Dataset/Cattle Dentition Dataset")
output_folder = "Cattle_DataSet_4_Classes/train"
model_weights = os.path.join(home_folder, "RESULTS/MODEL_FILES/best.pt")

predict_images(input_folder, output_folder, model_weights)
